# Crystal Cascade
A Sierra Nevada/Little Creatures inspired American Pale Ale

## Recipe

### Malt Bill:
 - Pilsner Malt (?EBC)
 - Australian Ale Malt (?EBC)
 - Crystal Malt (?EBC)

### Hops:
 - Cascade (?Alpha Acids)

### Calculate Water amounts
Need functions to take in batch size and work out the amounts all the way to strike water, inculding assumptions for losses

In [9]:
batch_size =  5 # input("Batch Size (L) = ")

Batch Size (L) = 4.5


In [ ]:
# This will be a module (file)
class constants(object):
    # Approximate rate of evaporation at boil (L/hr)
    evap_rate = ????????????
    # Approximate loss of volume due to grain absorption
    grain_disp = ????????????

In [ ]:
def strike_water():
    pass
def pre_boil_volume():
    pass
def post_boil_volume():
    return pre_boil_volume() - evaporation(time)
def evaporation(time=1):
    """ Returns evaporation (L) for a given boil time (hrs) """
    return time * constants.evap_rate